# Conversation Buffer Window
---

`ConversationBufferWindowMemory` trabaja de forma similar a `ConversationBufferMemory` (vease conversation_buffer.ipynb), tambien mantiene una lista de interacciones de la conersacion a lo largo del tiempo.

La diferencia principal radica, en que se trabaja con un cierto numero de conversaciones (se denomina k). Solo utiliza las ultimas `k` interacciones. Esto pueder resultar util para mantener una "ventana" de las
interacciones mas recientes y evitar que el historial (buffer) crezca demasiado.

Exploremos la funcionalidad basica.

Importamos `ConversationBufferWindowMemory` desde langchain.memory de la siguiente forma:

In [3]:
from langchain.memory import ConversationBufferWindowMemory

Instanciaremos la memoria y agregaremos unas conversaciones a la misma.

In [5]:
# Atencion al parametro k=1
memory = ConversationBufferWindowMemory(k=1)

memory.save_context(
    inputs={"input": "hi"},
    outputs={"output": "what's up?"}
)

memory.save_context(
    inputs={"input": "not much you"},
    outputs={"output": "not much"}
)


Ahora usemos el metodo `load_memory_variables` para mostrar el historial de la conversacion

In [6]:
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

Aqui podemos observar que solo regreso la ultima conversacion y no todo lo que deberia contener, esto se debe al parametro k que definimos (k=1)

Hagamos otro ejemplo, pero, le pediremos que devuelva toda la lista de mensajes, recordemos que esto es util para cuando estamos usando con un modelo de "chat".

In [8]:
memory = ConversationBufferWindowMemory(
    k=1, # numero de mensajes en el buffer
    return_messages=True, #regresar los mensajes
)

memory.save_context(
    inputs={"input": "hi"},
    outputs={"output": "what's up?"}
)

memory.save_context(
    inputs={"input": "not much you"},
    outputs={"output": "not much"}
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='not much you', additional_kwargs={}, example=False),
  AIMessage(content='not much', additional_kwargs={}, example=False)]}

Como podemos notar, tenemos el mismo comportamiento, y solo nos regresa el "ultimo" (par de mensajes)

## Uso en un "chain"
---

Veamos un ejemplo, emplearemos este tipo de memoria en un "chain" de tipo ConversationChain

In [13]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

# llm 
llm = OpenAI(temperature=0)

# memory
memory = ConversationBufferWindowMemory(
    k=2, # establecemos k=2, para solo obtener las ultimas 2 interaccion de mensajes en memoria
)

conversation_with_summary = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
)


Enviemos el primer mensaje

In [14]:
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:



> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

Enviemos otro mensaje, hay que poner atencion en la conversacion

In [15]:
conversation_with_summary.predict(input="What's their issues?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: What's their issues?
AI:

> Finished chain.


" The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected."

Si enviamos un nuevo mensaje, observemos que pasa con la conversacion.

In [16]:
conversation_with_summary.predict(input="Is it going well?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: What's their issues?
AI:  The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected.
Human: Is it going well?
AI:



> Finished chain.


" Yes, it's going well so far. We've already identified the problem and are now working on a solution."

In [17]:
conversation_with_summary.predict(input="What's the solution?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's their issues?
AI:  The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected.
Human: Is it going well?
AI:  Yes, it's going well so far. We've already identified the problem and are now working on a solution.
Human: What's the solution?
AI:

> Finished chain.


" The solution is to reset the router and reconfigure the settings. We're currently in the process of doing that."

Aqui hay que notar que la primer conversacion ya no aparece, solo aparecen las ultimas 2, como lo definimos con el parametro k=2 en ConversationBufferWindowMemory

Asi podemos mantener el contexto de forma compacta.

**Nota:**
 
Cabe mencionar, que solo se mantienen las ultimas dos (k=2), pero si la respuesta del llm es extensa, aun asi podriamos alcanzar el limite del modelo.